In [2]:
from urllib.request import urlopen
import json
from datetime import datetime
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
%matplotlib inline

In [27]:
def get_recent_article(date_span=7):
    # initialize
    article_api   = "https://alis.to/api/articles/recent?limit=100&page=%d"
    page           = 1
    result          = []
    # データ取得
    while True:
        raw_article_data  = urlopen(article_api % page).read().decode("utf-8") 
        json_article_data  = json.loads(raw_article_data)                    
        articles_data        = json_article_data["Items"]
        result.extend(articles_data)
        # ループ終了のチェック
        today                 = datetime.now() 
        last_data_date    = datetime.fromtimestamp(int(result[-1]["published_at"]))
        if (today-last_data_date).days >= (date_span+1):
            return result
        else:
            page += 1

In [20]:
def get_like_count(article_id):
    likes_api          = "https://alis.to/api/articles/%s/likes"
    raw_like_data  = urlopen(likes_api % article_id).read().decode("utf-8") 
    json_like_data  = json.loads(raw_like_data)                              
    like_count        = json_like_data["count"]
    return like_count

In [21]:
def get_user_name(user_id):
    user_api          = "https://alis.to/api/users/%s/info"
    raw_user_data = urlopen(user_api % user_id).read().decode("utf-8")     
    user_data        = json.loads(raw_user_data) 
    name               = user_data["user_display_name"]
    return name

In [22]:
def make_data(articles):
    data = []
    for article in articles:
        title             = article["title"]
        user_id        = article["user_id"]
        article_id     = article["article_id"]
        user_name  = get_user_name(user_id)
        like_count   = get_like_count(article_id)
        date            = datetime.fromtimestamp(int(article["published_at"]))
        url               = "https://alis.to/%s/articles/%s" % (user_id,article_id)
        data.append([title,url,user_name,like_count,date])
    return data

In [23]:
def make_rankig(data):
    ranking   = pd.DataFrame(data,columns=["titel","url","autehr","like","date"])
    not_old   = ranking["date"].map(lambda x:x.day) != ranking["date"].min().day
    not_new = ranking["date"].map(lambda x:x.day) != ranking["date"].max().day
    ranking   = ranking[not_old & not_new]
    ranking   = ranking.sort_values("like",ascending=False)
    return ranking

In [26]:
articles = get_recent_article(date_span=1)
data     = make_data(articles)
ranking  = make_rankig(data)
ranking

,titel,url,autehr,like,date
88,しまりす駆除騒ぎ,https://alis.to/shimaris/articles/azDVoEM96Ykg,しまりす,66,2018-08-26 14:29:07
107,毛根侍2「15話ペペ」,https://alis.to/moukonjoshi/articles/3656MBgnbdkz,毛根女子,46,2018-08-26 07:26:20
47,【予言】ビットコインＥＴＦのためにTetherは終わる！,https://alis.to/MEKAMEKA/articles/3k9JM4px9ByO,MEKAMEKA,42,2018-08-26 23:32:53
96,ジャンル開放して得意分野で記事を書きたくないですか？,https://alis.to/key-s2/articles/2jDnDQn1Yn5O,きぃ。,41,2018-08-26 11:33:57
56,Tasteemの現状で思ったこと。,https://alis.to/IBUKICHI/articles/azDVoQ8RqmVA,IBUKICHI（マイキチ）,37,2018-08-26 21:50:13
64,まだ豆乳で美肌になろうとしてるの？,https://alis.to/yoshidakunsansan/articles/3Vjo...,よしだ,33,2018-08-26 21:11:43
92,世界基軸通貨は失敗する,https://alis.to/umekichi/articles/a1ZpOl84Lv1V,うめ吉,31,2018-08-26 12:04:22
78,仮想通貨体験記 〜白崎ミキトの場合〜 第46話：過去,https://alis.to/Kaichi/articles/3PYow1PZRzzz,かいち＠漫画家,29,2018-08-26 17:32:47
63,ALISは、運営チームが一番興味深い件ｗ[コミュニティとして見たときのALIS],https://alis.to/torinohito/articles/3D4OQnPj7Q4x,torinohito,27,2018-08-26 21:17:02
108,QASHは仮想通貨取引所の未来を切り開くのか考察しました,https://alis.to/sasuke0025/articles/K5x5XN80wVVn,「君MONA！！」,27,2018-08-26 06:37:39
